# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

A newer version of GraphLab Create (v1.8) is available! Your current version is v1.7.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [35]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [36]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [37]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [40]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 0.325609     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 0.352088     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [41]:
model_all['coefficients'].show()


Canvas is updated and available in a tab in the default browser.


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

In [42]:
['bathrooms' , 'sqft_living' , 'sqft_living_sqrt' , 'grade' , 'sqft_above']

['bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above']

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [43]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [53]:
import numpy as np
minsum_rss = 725776517727024;
min_l1 = 0;

for l1 in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                  validation_set=None, verbose = False,
                                                  l2_penalty=0., l1_penalty=l1)
    expect = model.predict(validation)
    rss = ((validation['price'] - expect)**2).sum();
    if minsum_rss > rss:
        minsum_rss = rss
        min_l1 = l1
    print 'rss: %d l1:%d' % (rss, l1)
print 'best_rss:%d best_l1:%d' % (minsum_rss, min_l1)

rss: 625766285142460 l1:10
rss: 625766285362394 l1:31
rss: 625766286057885 l1:100
rss: 625766288257224 l1:316
rss: 625766295212186 l1:1000
rss: 625766317206080 l1:3162
rss: 625766386760658 l1:10000
rss: 625766606749278 l1:31622
rss: 625767302791634 l1:100000
rss: 625769507643885 l1:316227
rss: 625776517727024 l1:1000000
rss: 625799062845466 l1:3162277
rss: 625883719085425 l1:10000000
best_rss:625766285142460 best_l1:10


In [54]:
best_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                  validation_set=None, verbose = False,
                                                  l2_penalty=0., l1_penalty=min_l1)
test_rss = ((best_model.predict(testing) - testing['price'])**2).sum()
print test_rss


1.56983602382e+14


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [55]:
best_model.show()
best_model['coefficients'].show()
3162
625766317206080


Canvas is updated and available in a tab in the default browser.
Canvas is updated and available in a tab in the default browser.


625766317206080

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [56]:
17

17

In [57]:
best_model.predict(validation) - validation['price']


dtype: float
Rows: 9635
[117616.49514441582, 170861.959530783, 102966.88739580661, -87708.89137423068, 227236.3248703786, 52021.57093648764, 227726.11024307646, 84000.97905625403, 105463.02307220246, 126682.79562822921, -975324.2923875158, -265433.04888629727, -125905.52320499637, -28579.304429203272, 200905.02310891857, -168117.78457378736, -6279.5435020332225, -100722.36429470143, -74714.12473360985, -27292.93196084653, 42271.8522275798, -251854.34713585058, -77632.99493269937, -289523.9587590704, -263634.4768415695, 261753.73736475676, 261183.53607984388, 125390.66589669464, 109957.59786334552, 63955.93140795245, -9133.408897407935, 334697.9946869643, 266451.0185553832, 54538.11439161922, 159795.88455301576, -5167.660078765941, 165980.33113986044, 162379.78537396237, 133844.92063239182, 53882.926138148, 223190.69888733444, 71121.49296002428, 192621.3952937997, -124807.38364389795, 194074.34398646036, -39335.59307934757, -363815.67961562495, 172712.36038277362, 73307.33305831568, -27

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [58]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [59]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [60]:
import numpy as np

max_nonzeros = 7
result = [];

searchlist = l1_penalty_values = np.logspace(8, 10, num=20)
for l1 in searchlist:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                  validation_set=None, verbose = False,
                                                  l2_penalty=0., l1_penalty=l1)
    expect = model.predict(validation)
    rss = ((validation['price'] - expect)**2).sum();

    nnz = model['coefficients']['value'].nnz();
    if nnz == max_nonzeros:
        result += l1
        
    print 'rss: %d l1:%d nnz:%d' % (rss, l1, nnz)

rss: 627492659875075 l1:100000000 nnz:18
rss: 628210516770747 l1:127427498 nnz:18
rss: 629176689541061 l1:162377673 nnz:18
rss: 630650082718674 l1:206913808 nnz:18
rss: 632940229286984 l1:263665089 nnz:17
rss: 636268140229529 l1:335981828 nnz:17
rss: 641261198311187 l1:428133239 nnz:17
rss: 648983455376337 l1:545559478 nnz:17
rss: 660962217696068 l1:695192796 nnz:17
rss: 677261520727858 l1:885866790 nnz:16
rss: 701046815867076 l1:1128837891 nnz:15
rss: 737850622829047 l1:1438449888 nnz:15
rss: 796163109639833 l1:1832980710 nnz:13
rss: 869018172893693 l1:2335721469 nnz:12
rss: 966925692362085 l1:2976351441 nnz:10
rss: 1081867592324112 l1:3792690190 nnz:6
rss: 1244927360324958 l1:4832930238 nnz:5
rss: 1384161490235110 l1:6158482110 nnz:3
rss: 1230794720455647 l1:7847599703 nnz:1
rss: 1229157160638600 l1:10000000000 nnz:1


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [62]:
l1_penalty_min = 2976351441  
l1_penalty_max = 3792690190 

# answer : l1_penalty_min = 2796000000
#           l1 max  =   3793000000

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [63]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [74]:
result = []
for l1 in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                  validation_set=None, verbose = False,
                                                  l2_penalty=0., l1_penalty=l1)
    expect = model.predict(validation)
    rss = ((validation['price'] - expect)**2).sum();

    nnz = model['coefficients']['value'].nnz();
    if nnz == 7:
        result.append((rss, l1))
        
    print 'rss: %d l1:%d nnz:%d' % (rss, l1, nnz)
    
print result

rss: 1081867592324112 l1:3792690190 nnz:6
rss: 1077632775581415 l1:3749724992 nnz:6
rss: 1073504549585599 l1:3706759795 nnz:6
rss: 1069464335425585 l1:3663794598 nnz:6
rss: 1065707689498230 l1:3620829400 nnz:6
rss: 1060799531763286 l1:3577864203 nnz:7
rss: 1055992735342999 l1:3534899006 nnz:7
rss: 1051147625612862 l1:3491933808 nnz:7
rss: 1046937488751711 l1:3448968611 nnz:7
rss: 1043237237871701 l1:3406003414 nnz:8
rss: 1038554735941040 l1:3363038216 nnz:8
rss: 1034616909232827 l1:3320073019 nnz:8
rss: 1028247992205977 l1:3277107822 nnz:10
rss: 1018298780553819 l1:3234142624 nnz:10
rss: 1008477167020094 l1:3191177427 nnz:10
rss: 998783211265890 l1:3148212230 nnz:10
rss: 989328342459473 l1:3105247032 nnz:10
rss: 981188367942453 l1:3062281835 nnz:10
rss: 974019450084554 l1:3019316638 nnz:10
rss: 966925692362085 l1:2976351441 nnz:10
[(1060799531763286.9, 3577864203.4210525), (1055992735342999.4, 3534899006.1052632), (1051147625612862.5, 3491933808.7894735), (1046937488751711.5, 344896861

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [69]:
l1 = 3448968611
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                  validation_set=None, verbose = False,
                                                  l2_penalty=0., l1_penalty=l1)

rss = 1046937488751711 
model['coefficients'].print_rows(num_rows=19)


+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None | 222253.192544 |
|     bedrooms     |  None | 661.722717782 |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 15873.9572593 |
|   sqft_living    |  None | 32.4102214513 |
| sqft_living_sqrt |  None | 690.114773313 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 2899.42026975 |
|    sqft_above    |  None | 30.0115753022 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x

In [ ]:
[bedrooms , bathrooms  , ]